Importando Bibliotecas:

In [ ]:
# APS CC 6 Semestre UNIP
# Integrantes: 
# Davi Ribeiro - F21AFG6
# Ryan Bruno de Souza - N5664H4
# Ryan Vitor Rodrigues Teixeira - N662BC4
# Victor Hugo Souza Ferreira - N587FF6
!pip  install numpy
!pip install pathlib


In [2]:
import numpy as np # O Uso do Numpy será obrigatório para conseguirmos usar as bibliotecas


from pathlib import Path
import os

Carregando Diretórios:

In [ ]:
dataset_folder = Path('dataset_aps')
os.listdir(dataset_folder)

In [ ]:
path_labels = {
    'a' : Path('a_l'),
    'e' : Path('e_l'),
    'i' : Path('i_l'),
    'o' : Path('o_l'),
    'u' : Path('u_l'),
    'A' : Path('A_u'),
    'E' : Path('E_u'),
    'I' : Path('I_u'),
    'O' : Path('O_u'),
    'U' : Path('U_u')
}

os.listdir(dataset_folder / path_labels['a'])

In [4]:
label_to_file = {
    k: [dataset_folder / pl/ f for f in os.listdir(dataset_folder / pl)] for k, pl in path_labels.items()
}

label_to_file['a'][0]

WindowsPath('dataset_aps/a_l/train_61_00000.png')

### Lendo Imagens:

In [ ]:
!pip install matplotlib


In [3]:
from matplotlib.image import imread

In [6]:
img = imread(label_to_file['a'][0])
print(type(img))

<class 'numpy.ndarray'>


In [7]:
bin_matrix = []
for linha in img:
    bin_matrix.append([celula[0] for celula in linha])

np.array(bin_matrix)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [8]:
def print_bin_matrix(img: np.array):
    matrix = ''
    for linha in img:
        for p in linha:
            matrix += f'{int(p):>1}'
        matrix += '\n'
    print(matrix)

In [ ]:
print_bin_matrix(bin_matrix)

### Salvando Dataset em formato ML:

In [29]:
X = []
Y = []
for key, files in label_to_file.items():
    for path in files:
        Y.append(key)
        image = imread(path)
        matrix = []
        for linha in image:
            matrix.append([int(celula[0]) for celula in linha])

        X.append(np.array(matrix))

In [31]:

# transformando matriz da imagem em array de uma dimensao
X = [x.reshape(1, -1) for x in X]

In [39]:
X = np.array(X)

In [43]:
# função que corrige o array estar compactado de forma errada
def process_X(X: np.array):
    matrix = []
    for item in X:
        for subitem in item:
            matrix.append(subitem)
    matrix = np.array(matrix)
    return matrix

In [44]:
X = process_X(X)

In [45]:
X

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [46]:
# transformando arary de labels em binarios (se eh i ou nao)
Y = np.array(Y)

In [47]:
Y = ((Y=='i')|(Y=='I'))

In [ ]:
Y

In [ ]:
!pip install sklearn


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

In [50]:
# split entre 0, 1
# separando 20% para teste
split_test_threshold = 0.2

selection_iter = StratifiedShuffleSplit(n_splits=1, test_size=split_test_threshold, random_state=42)

In [51]:
train_index, test_index = next(selection_iter.split(X, Y))

In [52]:
X_train = X[train_index]
X_test = X[test_index]
y_train = Y[train_index]
y_test = Y[test_index]

In [53]:
print(len(Y))
print(len(y_train))

900
720


### Salvando os Dados:

In [54]:
with open('train_index.npy', 'wb') as f:
    np.save(f, train_index)

In [55]:
with open('test_index.npy', 'wb') as f:
    np.save(f, test_index)

In [56]:
with open('X.npy', 'wb') as f:
    np.save(f, X)

In [57]:
with open('y.npy', 'wb') as f:
    np.save(f, Y)

In [58]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [59]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

In [60]:
with open('y_train.npy', 'wb') as f:
    np.save(f, y_train)

In [61]:
with open('y_test.npy', 'wb') as f:
    np.save(f, y_test)

### Carregando dados

In [5]:
with open('train_index.npy', 'rb') as f:
    train_index = np.load(f)

In [6]:

with open('test_index.npy', 'rb') as f:
    test_index = np.load(f)

In [7]:

with open('X.npy', 'rb') as f:
    X = np.load(f)

In [8]:
with open('y.npy', 'rb') as f:
    y = np.load(f)

In [9]:

with open('X_train.npy', 'rb') as f:
    X_train = np.load(f)

In [10]:
with open('X_test.npy', 'rb') as f:
    X_test = np.load(f)

In [11]:
with open('y_train.npy', 'rb') as f:
    y_train = np.load(f)

In [12]:

with open('y_test.npy', 'rb') as f:
    y_test = np.load(f)

### Treinando Classificador Binário

# SGDClassifier

In [ ]:
!pip install sklearn

In [18]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

Testando Classificador Binário

In [72]:
y_test[0]

False

In [74]:
sgd_clf.predict([X_test[0]])

array([False])

### Acurácia Média

In [81]:
def acuracia_media(x_test: np.array, y_test: np.array):
    size = len(x_test)
    acertos = 0
    for predict, real in zip(sgd_clf.predict(X_test), y_test):
        if predict == real:
            acertos += 1
    acuracia = acertos/size
    return acuracia

In [84]:
# acuracia_media somente com apenas um mesmo treino
acuracia = acuracia_media(X_test, y_test)
acuracia

0.9722222222222222

In [17]:
# acuracia media usando cross_val_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [20]:
acc = cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring='accuracy')
acc

array([0.84166667, 0.95833333, 0.9375    ])

In [16]:
def media(acc: np.array):
    counter = 0
    media = 0
    for num in acc:
        counter += 1
        media += num
    media = media / counter
    return media

In [21]:
media(acc)

0.9125

### Matrix de Confusão

In [92]:
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)
confusion_matrix(y_train, y_train_pred)

array([[525,  51],
       [ 12, 132]], dtype=int64)

verdadeiros positivos: 132

falsos positivos: 51

verdadeiros negativo: 525

falsos negativo: 12

### Precision:

In [93]:
precision_score(y_train, y_train_pred)

0.7213114754098361

### Recall:

In [94]:
recall_score(y_train, y_train_pred)

0.9166666666666666

### F1:

In [95]:
f1_score(y_train, y_train_pred)

0.8073394495412843

# Random Forest Classifier

In [1]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
rf = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
rf.fit(X_train, y_train)

### acurácia média

In [25]:
acc = cross_val_score(rf, X_train, y_train, cv=3, scoring='accuracy')
media(acc)

0.8138888888888888

### Matrix de Confusão

In [26]:
y_train_pred = cross_val_predict(rf, X_train, y_train, cv=3)
confusion_matrix(y_train, y_train_pred)

array([[576,   0],
       [134,  10]], dtype=int64)

Verdadeiro Positivo: 10

Falso Positivo: 0

Verdadeiro Negativo: 576

Falso Negativo: 134

### Precision

In [27]:
precision_score(y_train, y_train_pred)

1.0

### Recall

In [28]:
recall_score(y_train, y_train_pred)

0.06944444444444445

### f1

In [29]:
f1_score(y_train, y_train_pred)

0.12987012987012989

### Salvando modelo treinado

In [109]:
from joblib import dump, load

In [ ]:
sgd_clf.fit(X, y)

In [110]:
dump(sgd_clf, 'modelo_salvo.joblib')

['modelo_salvo.joblib']

# Conclusão:

## SGDClassifier:
### Acurácia média: 91,25%
### Precision: 72,13%
### Recall: 91,66%
### f1: 80,73%

## Random Forest Classifier:
### Acurácia média: 81,38%
### Precision: 100%
### Recall: 6,94%
### f1: 12,98%

Baseado nas Pontuações de f1 podemos definir que para este caso o SGDClassifier se sai muito melhor do que o Random Forest Classifier.